<a href="https://colab.research.google.com/github/nathalyAlarconT/GenAI_Workshops/blob/main/Basic_Agent_Crewai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --q crewai crewai-tools
!pip install --q langchain_google_genai

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GoogleAIStudio')

# GOOGLE_API_KEY = ""

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
# os.environ["OPENAI_API_KEY"] = userdata.get('openai_key')

In [4]:
from crewai import Agent, Task, Crew, LLM

https://docs.crewai.com/concepts/tools

In [5]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

In [6]:
my_llm = LLM(
              model='gemini/gemini-2.0-flash',
              api_key=GOOGLE_API_KEY
            )

In [7]:
# from langchain_google_genai import ChatGoogleGenerativeAI

# os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
# my_llm2 = ChatGoogleGenerativeAI(model="gemini/gemini-2.0-flash",
#                              verbose = True,
#                              temperature = 0.6,
#                              google_api_key=GOOGLE_API_KEY)

In [8]:
company_site_documentation_url = "https://codoid.com/ai/cursorai-vs-copilot-a-detailed-analysis/"

## Agents Definition

In [9]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at {company_name} and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	llm = my_llm,
	allow_delegation=False,
	verbose=True
)

In [10]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at {company_name} and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
  llm = my_llm,
	verbose=True
)

## Tools Definition

In [11]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url = company_site_documentation_url,
)

In [12]:
text = docs_scrape_tool.run()
print(text)

Using Tool: Read website content
 Cursor AI vs Copilot: A Detailed Analysis - Codoid Home About Us Services Testing Services Mobile App Testing Our mobile testing strategic roadmap will maximize your reward and minimize product risk. Mobile app testing needs continuously testing the apps for functionality, performance… Automation Testing With the focus on Automation testing, we work on various automation testing services for web, mobile, desktop, game, car infotainment systems, and Mixed reality applications. Manual Testing Although Codoid delivers the best automated testing available, our manual testing services offer increased debugging. Sometimes you can’t find bugs without a fine-tooth comb. Our manual testing is as fine as it gets. Performance Testing We are a performance testing services company with robust technical and business prowess and can guarantee smooth working of your business. Security Testing Achieve unparalleled security for your systems and protect your business fro

## Tasks Definition

In [13]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

In [14]:
# QA will only review the documentation provided by the support agent
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


# Crew Definition

In [16]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=True,


)

In [17]:
inputs = {
    "company_name": "AI IDEs Copilot",
    "customer": "Coderoad",
    "person": "Nathaly",
    "inquiry": "I want to know how to enable autocomplete on the IDE "
}
result = crew.kickoff(inputs=inputs)

# Agent: Senior Support Representative
## Task: Coderoad just reached out with a super important ask:
I want to know how to enable autocomplete on the IDE 

Nathaly from Coderoad is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


# Agent: Senior Support Representative
## Thought: Okay, I understand. Nathaly from Coderoad wants to know how to enable autocomplete in the IDE. I need to provide a detailed and informative response. Since I don't have built-in documentation or specific tools for Copilot settings, I will leverage the "Read website content" tool to access a comparison article between Cursor AI and Copilot. This article might contain information about Copilot's features and settings, including autocomplete.
## Using tool: Read website content
## Tool Input: 
"{}"
## Tool Output: 
 Cursor AI vs Copilot: A Detailed Analysis - Codoid Home Abo

In [21]:
from IPython.display import Markdown
Markdown(result.raw)

Hi Coderoad,

Thanks for reaching out! We're always happy to help.

Regarding enabling autocomplete in Copilot, it should be enabled by default once you have the GitHub Copilot plugin installed and activated in your IDE (such as Visual Studio Code, Visual Studio, JetBrains IDEs, or Neovim). Copilot provides code suggestions in real-time and can auto-complete text as you type, making your coding process smoother and faster.

If you're not seeing autocomplete suggestions, here are a few things you can check. Let's troubleshoot this together:

1.  **Plugin Installation:** Double-check that the GitHub Copilot plugin is properly installed and enabled in your IDE.
    *   **For VS Code:** Go to the Extensions view (Ctrl+Shift+X or Cmd+Shift+X). Search for "GitHub Copilot." Ensure it's installed and enabled. If it's disabled, click the "Enable" button.
    *   **For JetBrains IDEs (IntelliJ, PyCharm, etc.):** Go to File > Settings (or Preferences on macOS) > Plugins. Search for "GitHub Copilot." Ensure the checkbox next to it is checked.
2.  **Activation/Login:** Make sure you're logged in to your GitHub account *within* your IDE and that Copilot is activated.
    *   You might need to authenticate your account. Usually, Copilot will prompt you to log in when you first install it. If not, look for a Copilot icon in your IDE's status bar or toolbar and click it to log in.
    *   Verify your GitHub account has an active Copilot subscription. You can check this on your GitHub account settings under "Copilot".
3.  **IDE Settings:** Some IDE settings can inadvertently interfere with autocomplete.
    *   **VS Code:** Go to File > Preferences > Settings (or Code > Preferences > Settings on macOS). Search for "editor.suggest" and make sure `editor.suggest.enabled` is set to `true`. Also, check for any settings that might disable suggestions based on file type.
    *   **JetBrains IDEs:** Go to File > Settings (or Preferences on macOS) > Editor > General > Code Completion. Ensure that "Autopopup code completion" is enabled.
4.  **Copilot Settings:** Explore Copilot-specific settings within your IDE.
    *   **VS Code:** Go to File > Preferences > Settings and search for "GitHub Copilot". Here, you can adjust settings like the number of suggestions Copilot provides.
    *   **JetBrains IDEs:** Look for Copilot-related settings under File > Settings (or Preferences on macOS) > Editor > Code Completion, or by searching "Copilot" in the settings.
5.  **File Type:** Autocomplete functionality depends on the file type you're working with. Copilot officially supports languages like Python, JavaScript, TypeScript, JSX, Go, C#, C++, PHP, Ruby, Swift, Scala, and Shell. Plain text files or less common languages might not trigger autocomplete.
6.  **Check for Errors:** Keep an eye on your IDE's console or output window for any error messages or warnings related to Copilot. These messages can provide valuable clues.
    *   **VS Code:** View > Output (select "GitHub Copilot" from the dropdown).
    *   **JetBrains IDEs:** View > Tool Windows > Event Log.
7.  **Restart IDE:** A classic troubleshooting step, but often effective! Close and reopen your IDE to ensure all plugins and services are properly loaded.
8. **Check Proxy Settings:** If you are behind a corporate network, Copilot might have trouble connecting to the internet. Check your IDE's proxy settings to ensure they are correctly configured.
    *   **VS Code:** Go to File > Preferences > Settings and search for "proxy".
    *   **JetBrains IDEs:** Go to File > Settings (or Preferences on macOS) > Appearance & Behavior > System Settings > HTTP Proxy.
9. **Copilot Status:** Check Copilot's status page to see if there are any known outages or issues: [https://www.githubstatus.com/](https://www.githubstatus.com/)

If you've checked all of these things and autocomplete is still not working, please provide the following information so I can further assist you:

*   Which IDE are you using? (e.g., VS Code, PyCharm, etc.)
*   What programming language are you working with? (e.g., Python, JavaScript, etc.)
*   Are there any error messages or warnings in the IDE's console? Please copy and paste the full error message.
*   What is your operating system? (Windows, macOS, Linux)
*   What Copilot plan are you using? (Individual, Business, Enterprise)

For more detailed information, you can also refer to the official GitHub Copilot documentation: [https://docs.github.com/en/copilot](https://docs.github.com/en/copilot)

I hope this helps get you up and running! Let me know how it goes or if you have any other questions. We're here to help!

Best regards,

Your Senior Support Representative